# Adressage indirect et indexé

Jusqu'ici, en utilisant `LDR` et `STR` pour accéder à la mémoire, nous nous servions d'un «adressage direct» - nous précisions *directement* l'adresse du mot mémoire soit par un nombre (en décimal ou en hexadécimal), soit par une étiquette qui est traduite en  un nombre par l'assembleur avant exécution:

- `LDR R0, 100`: charge dans R0 le contenu du mot mémoire situé à l'adresse 100,
- `STR R1, 0xf4`: range en mémoire le contenu du registre R1 dans le mot mémoire situé à l'adresse 0xf4, (hex)
- `LDR R2, uneEtiquette`: charge dans R2 le contenu du mot mémoire situé à l'adresse étiquetée par `uneEtiquette` dans le code.

L'exercice du pendu (chapitre 4) a dû vous faire sentir les limitations de l'adressage direct. Avec ce mode d'adressage, il est impossible de généraliser la plupart des routines: même pour dessiner une simple ligne droite à l'écran, vous êtes obligé d'écrire une instruction pour chaque pixel.

À présent, observez attentivement le code ci-dessous (commentaires inclus). Notez en passant que nous passons du mode basse résolution du chapitre précédent au mode moyenne résolution: une grille de 64 x 48 pixels, chaque pixel étant toujours représenté par un mot (4 octets) qui contient une couleur en utilisant 24 bits (8\*3):

```
    MOV R1, #.PixelScreen  // adresse du début de la mémoire graphique de résolution moyenne
    MOV R2, #.red          
    MOV R3, #0             // Notre «compteur» (index) de pixel
loop:
    ADD R4, R1, R3         // R4 contient l'adresse du pixel courant
    STR R2, [R4]           // Colorie le pixel d'adresse R4
    ADD R3, R3, #4         // Incrémente notre compteur (4 octets = 1 mot = 1 pixel)
    CMP R3, #256           // 64*4=256 qui correspond donc au premier pixel de la deuxième ligne
    BLT loop
    HALT
```

L'idée principale et nouvelle se situe dans l'instruction `STR R2, [R4]`. L'utilisation des crochets \[ *square brackets* \] précise que la deuxième opérande utilise l'**adressage indirect** ce qui signifie que nous donnons l'adresse **indirectement** (ni un nombre, ni une étiquette) en utilisant dans ce cas le contenu d'un registre (lequel contient l'adresse d'un mot mémoire). 

`STR R2, [R4]` ne signifie **pas**: «ranger le contenu de R2 dans R4» (ce qui ne serait de toute façon pas possible avec une instruction `STR`). Cela signifie: «ranger le contenu de R2 à l'*adresse mémoire détenue par R4*».

Un autre nom pour une adresse indirecte est **pointeur**. Les pointeurs sont utilisées de manière intensive, par exemple dans les implémentations bas niveau de toutes les principales structures de données (tableaux, liste, pile, file, arbre ...).

#### Exercice 26

Tester le code donné plus haut. Il devrait afficher une ligne rouge à travers la zone d'affichage. Modifier le pour peindre en rouge la 10e ligne et prendre une capture d'écran du résultat.
___

### Un peu d'histoire - John Von Neumann ...

Dans l'exemple précédent, l'adresse indirecte, détenue par R4, est produite sur la base d'une valeur constante dans R1 (l'adresse de départ de la grille de pixels) à laquelle on ajoute un nombre variable (en R3). Nous pourrions dire que la valeur de R1 est l'adresse de **base** et celle de R3 est un **index** variable, ajouté à la base. Dans cette situation, qui est très commune, il existe une syntaxe simplifiée connue sous le nom d'**adressage indexé**, qui (sur l'ARMlite) est une forme spécialisée de l'adressage indirect:

```
    MOV R1, #.PixelScreen  //adresse du début de la mémoire graphique de résolution moyenne - CONSTANTE
    MOV R2, #.red          
    MOV R3, #0             //Notre «compteur» ou index
loop:
    STR R2, [R1+R3]        //Colorie le pixel situé à l'adresse définie par R1+R3
    ADD R3, R3, #4         //Incrémente notre compteur (4 octets = 1 mot = 1 pixel)
    CMP R3, #256           
    BLT loop
    HALT
```

L'adressage indexé permet ici d'éliminer une instruction et le besoin d'un registre supplémentaire (R4 dans la première version). Les crochets autour de R1+R3 sont toujours nécessaires ici - car l'écriture R1+R3 n'est ni un registre ni une valeur connue de l'assembleur au moment de la traduction (avant l'exécution), mais celle d'une adresse indirecte.

#### Exercice 27

Tester cette nouvelle version du code (utilisant l'adressage indexé) et vérifier par vous-même qu'il produit le même effet.

Modifiez alors le code de façon à dessiner une ligne verticale verte démarrant au pixel de gauche le plus haut. Souvenez-vous qu'il y a maintenant 64 pixels par ligne (et non 32) et qu'il y a toujours 4 octets (= 1 mot) par pixel. Prendre une capture d'écran montrant le nouveau code et le résultat.

Enfin, écire du code de façon à dessiner un rectangle bleu (plein) de 20 pixels de large pour 10 pixels de haut et démarrant au pixel de gauche le plus haut comme précédemment. Vous aurez besoin de deux registres - pour suivre les coordonnées en x et en y  - qu'il faudra combiner dans un troisième (registre) de manière à former l'index à ajouter à la base en R1 pour dessiner chaque pixel. Commenter votre code. Prendre une capture d'écran montrant le nouveau code et le résultat.

____

L'adressage indexé est le moyen par lequel, un langage de haut niveau (comme Python), implémente les tableaux (ou liste), en permettant la récupération d'un élément arbitraire de celui-ci en temps constant - O(1) - plutôt que dans un temps proportionnel à la taille du tableau - 0(n).

## Implémentation du trie bulle en utilisant l'adressage indexé

Dans l'exemple qui suit, nous utilisons l'**adressage indexé** afin de trier un tableau de nombres qui forme une suite de données (consécutives) en mémoire. Nous utiliserons l'algorithme du «trie bulle» parce qu'il est simple à écrire et à observer en exécutant le programme lentement. Sachez néanmoins que c'est l'un des algorithmes de trie les moins performants.

```
// Définition et initialisation des registres
    MOV R0, #tableau      //base
    LDR R1, tailleTableau //taille en mots
    LSL R1, R1, #2        //taille en octets (*4) ; l'index du dernier elt du tableau est R1 - 4
    MOV R2, #0            //initialisation du compteur de boucleExterne   
// R3: index du premier de la paire de boucleInterne
// R4: index limite haute de boucleInterne (R3 < R4)
// R5: index du second de la paire de boucleInterne
// R6: premier de la paire
// R7: second de la paire
// R8: ?

debutDeBoucleExterne: 
//sert à délimiter le sous-tableau (0-R4) manipulé par boucleInterne
    MOV R3, #0 // R3 va parcourir 0,4,...R4
    SUB R4, R1, R2
    SUB R4, R4, #4 // tailleTableau - 4, tailleTableau - 8, ..., 
    MOV R8, #0

boucleInterne: 
//récupère deux éléments consécutifs, les compare et les échange si besoin
// au final, le plus grand elt du sous-tableau est son dernier elt
    LDR R6, [R0+R3]
    ADD R5, R3, #4
    LDR R7, [R0+R5]
    CMP R6, R7
    BGT echange
    B suiteBoucleInterne

echange:
    STR R7, [R0+R3]
    STR R6, [R0+R5]
    MOV R8, #1

suiteBoucleInterne:
    ADD R3, R3, #4
    CMP R3, R4
    BLT boucleInterne

suiteBoucleExterne:
    CMP R8, #0
    BEQ fait
    ADD R2, R2, #4
    CMP R2, R1
    BGT fait
    B debutDeBoucleExterne

fait: HALT

.ALIGN 256
tailleTableau: 10
tableau: //peuplé à la main ...
    9
    8
    7
    6
    5
    4
    3
    2
    1
    0
```

    9 8 7 ... 1 0  -> 8 9 7 ... 1 0 ->  8 7 9 ... 1 0 -> ... -> 8 7 6 ... 9 0 -> 8 7 6 ... 0 9   
    ↑ ↑          |      ↑ ↑        |        ↑ ↑      |                    ↑ ↑|   ↑ ↑        | 
    
    -> ... -> 7 6 5 ... 8 9 -> ... -> 1 0 2 ... 8 9 -> 0 1 2 ... 8 9 (trié)
              ↑ ↑      |              ↑ ↑| 

#### Exercice 28

Soumettre le programme à l'ARMlite sans le faire tourner. Inspecter les données en mémoire à partir de l'adresse 0x00100. Faites ensuite tourner le programme et inspecter à nouveau les données: elles ont été triées dans l'ordre croissant de leur valeur.

À présent, modifier les données du tableau en utilisant des nombres aléatoires à plusieurs chiffres décimaux. En ajouter quelques-unes à la fin tout en ajustant la valeur de `tailleTableau`.

Soumettre votre programme et prendre des captures d'écran partielles montrant le tableau en mémoire avant exécution et après.

Dans la partie `// Définition et initialisation des registres` vous verrez `R8: ?`. Votre mission est d'identifier le rôle et la signification de ce registre.

Commencer par relever toutes les lignes de code où R8 est référencé. Ensuite, suivre son évolution, soit sur papier, soit en faisant tourner le programme lentement et/ou en mode pas à pas, afin de comprendre le rôle de ce registre. Trouver alors l'avantage que son utilisation confère à l'algorithme.

## Implémentation de la recherche dichotomique \[ *binary search* \] en utilisant l'adressage indirect

Voici, «tout de go», l'implémentation de cet algorithme en utilisant l'**adressage indirect**:

```
// Rôle des registres
  //R0: valeur cible
  //R1: pointeur borne basse
  //R2: pointeur milieu
  //R3: pointeur borne haute
  //R4: valeur temporaire
  //R5: utiliser pour afficher les messages

debut:
    MOV R1, #premier
    MOV R3, #dernier
    MOV R5, #msg1
    STR R5, .WriteString
    LDR R0, .InputNum
    STR R0, .WriteUnsignedNum

boucle:
    ADD R2, R1, R3
    LSR R2, R2, #3 // Diviser par 8
    LSL R2, R2, #2 // puis multiplier par 4. Au final, on a divisé par 2 en s'assurant que l'adresse est un mutiple de 4.
    LDR R4, [R2]   // récupérer la valeur du milieu
    CMP R0, R4     // comparaison avec la valeur cible
    BEQ trouve
    BLT sousMilieu
    // on doit être au dessus du milieu ici
    MOV R1, R2
    ADD R1, R1, #4 //modification de la borne basse
    B verifSuperposition
    
sousMilieu:
    MOV R3, R2
    SUB R3, R3, #4
    B verifSuperposition

verifSuperposition:
    CMP R1, R3
    BGT pasTrouve
    B boucle

pasTrouve:
    MOV R5, #msg3
    STR R5, .WriteString
    B debut

trouve:
    MOV R5, #msg2
    STR R5, .WriteString
    STR R2, .WriteHex
    B debut

msg1: .ASCIZ "\nRecherche de ?"
msg2: .ASCIZ "\na pour adresse en mémoire: "
msg3: .ASCIZ "\nJe ne l'ai pas trouve!"

.ALIGN 256
premier: 3
    6
    7
    15
    22
    24
    31
    50
    79
dernier: 94
```

#### Exercice 29

Faites tourner ce programme puis réaliser une copie d'écran qui montre une recherche fructueuse en mettant en valeur le mot mémoire dont l'adresse est fournie en retour.

Montrer ensuite le résultat d'une recherche qui n'aboutit pas c'est-à-dire dont le nombre cible ne figure pas parmi les nombres du tableau.

Quelle hypothèse fait l'algorithme de recherche dichotomique à propos des données parmi lesquelles il effectue sa recherche?